In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
import scipy.io as sio
import numpy as np

file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_steps, features)
X = np.transpose(data_array, (2, 1, 0))

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, label_array, test_size=0.2, random_state=42)

# Define and train the LSTM model
lstm_model = Sequential([
    LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(units=1, activation='sigmoid')
])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

# Extract LSTM features from validation set
lstm_features = lstm_model.predict(X_val)

# Train ensemble classifiers
svm_classifier = SVC()
svm_classifier.fit(lstm_features, y_val)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(lstm_features, y_val)

# Define and train 1D CNN classifier
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=1, activation='sigmoid')
])
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Extract CNN features from validation set
cnn_features = cnn_model.predict(X_val)

# Combine features from LSTM and CNN
ensemble_features = np.concatenate([lstm_features, cnn_features], axis=1)

# Train ensemble classifiers on combined features
svm_classifier.fit(ensemble_features, y_val)
rf_classifier.fit(ensemble_features, y_val)

# Evaluate ensemble classifiers
ensemble_lstm_acc = accuracy_score(y_val, svm_classifier.predict(ensemble_features))
ensemble_rf_acc = accuracy_score(y_val, rf_classifier.predict(ensemble_features))

print("Ensemble LSTM + SVM Accuracy:", ensemble_lstm_acc)
print("Ensemble LSTM + Random Forest Accuracy:", ensemble_rf_acc)


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 27s 94ms/step - accuracy: 0.5737 - loss: 0.6904 - val_accuracy: 0.6014 - val_loss: 0.6792
Epoch 2/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 29s 104ms/step - accuracy: 0.6250 - loss: 0.6469 - val_accuracy: 0.5968 - val_loss: 0.6815
Epoch 3/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 29s 104ms/step - accuracy: 0.6606 - loss: 0.6187 - val_accuracy: 0.5819 - val_loss: 0.6821
Epoch 4/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 30s 108ms/step - accuracy: 0.6838 - loss: 0.5959 - val_accuracy: 0.5878 - val_loss: 0.6869
Epoch 5/5
277/277 ━━━━━━━━━━━━━━━━━━━━ 31s 110ms/step - accuracy: 0.6996 - loss: 0.5769 - val_accuracy: 0.5743 - val_loss: 0.6968
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Epoch 1/10


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5954 - loss: 2.5404 - val_accuracy: 0.6095 - val_loss: 0.6820
Epoch 2/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6235 - loss: 0.6772 - val_accuracy: 0.6095 - val_loss: 0.6738
Epoch 3/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6136 - loss: 0.6707 - val_accuracy: 0.6095 - val_loss: 0.6707
Epoch 4/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6151 - loss: 0.6672 - val_accuracy: 0.6095 - val_loss: 0.6699
Epoch 5/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6388 - loss: 0.6562 - val_accuracy: 0.6095 - val_loss: 0.6699
Epoch 6/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6257 - loss: 0.6614 - val_accuracy: 0.6095 - val_loss: 0.6701
Epoch 7/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6193 - loss: 0.6643 - val_accuracy: 0.6095 - val_loss: 0.6702
Epoch 8/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6270 - loss: 0.6605 - val_accuracy: 0.609

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Ensemble LSTM + SVM Accuracy: 0.6094808126410836
Ensemble LSTM + Random Forest Accuracy: 0.9990970654627539


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from imblearn.over_sampling import RandomOverSampler
import scipy.io as sio
import numpy as np

file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_steps, features)
X = np.transpose(data_array, (2, 1, 0))

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, label_array, test_size=0.2, random_state=42)

# Reshape y_train for RandomOverSampler
y_train_reshaped = y_train.reshape(-1)

# Apply RandomOverSampler to balance classes in training data
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train_reshaped)

# Reshape X_train_resampled back to original shape
X_train_resampled = X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2])

# Define and train the LSTM model
lstm_model = Sequential([
    LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(units=1, activation='sigmoid')
])
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Extract LSTM features from validation set
lstm_features = lstm_model.predict(X_val)

# Train ensemble classifiers
svm_classifier = SVC()
svm_classifier.fit(lstm_features, y_val)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(lstm_features, y_val)

# Define and train 1D CNN classifier
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=1, activation='sigmoid')
])
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Extract CNN features from validation set
cnn_features = cnn_model.predict(X_val)

# Combine features from LSTM and CNN
ensemble_features = np.concatenate([lstm_features, cnn_features], axis=1)

# Train ensemble classifiers on combined features
svm_classifier.fit(ensemble_features, y_val)
rf_classifier.fit(ensemble_features, y_val)

# Evaluate ensemble classifiers
ensemble_lstm_acc = accuracy_score(y_val, svm_classifier.predict(ensemble_features))
ensemble_rf_acc = accuracy_score(y_val, rf_classifier.predict(ensemble_features))

print("Ensemble LSTM + SVM Accuracy:", ensemble_lstm_acc)
print("Ensemble LSTM + Random Forest Accuracy:", ensemble_rf_acc)


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 45s 126ms/step - accuracy: 0.5125 - loss: 0.7140 - val_accuracy: 0.5300 - val_loss: 0.6959
Epoch 2/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 54s 157ms/step - accuracy: 0.6114 - loss: 0.6580 - val_accuracy: 0.5368 - val_loss: 0.6915
Epoch 3/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 54s 156ms/step - accuracy: 0.6674 - loss: 0.6287 - val_accuracy: 0.5404 - val_loss: 0.7004
Epoch 4/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 54s 156ms/step - accuracy: 0.7156 - loss: 0.5840 - val_accuracy: 0.5422 - val_loss: 0.7152
Epoch 5/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 55s 160ms/step - accuracy: 0.7515 - loss: 0.5446 - val_accuracy: 0.5323 - val_loss: 0.7468
Epoch 6/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 53s 153ms/step - accuracy: 0.7844 - loss: 0.4960 - val_accuracy: 0.5205 - val_loss: 0.7747
Epoch 7/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 52s 152ms/step - accuracy: 0.8153 - loss: 0.4448 - val_accuracy: 0.5359 - val_loss: 0.8112
Epoch 8/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 53s 152ms/step - accuracy: 0.8360 - loss: 0

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5237 - loss: 2.5147 - val_accuracy: 0.4786 - val_loss: 0.7700
Epoch 2/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6271 - loss: 0.6503 - val_accuracy: 0.5797 - val_loss: 0.7051
Epoch 3/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6898 - loss: 0.5872 - val_accuracy: 0.5752 - val_loss: 0.7149
Epoch 4/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7119 - loss: 0.5604 - val_accuracy: 0.5679 - val_loss: 0.7684
Epoch 5/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7403 - loss: 0.5280 - val_accuracy: 0.5869 - val_loss: 0.7816
Epoch 6/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7488 - loss: 0.5061 - val_accuracy: 0.5828 - val_loss: 0.7987
Epoch 7/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7850 - loss: 0.4617 - val_accuracy: 0.5661 - val_loss: 0.7323
Epoch 8/10
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7965 - loss: 0.4444 - val_accu

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Ensemble LSTM + SVM Accuracy: 0.618510158013544
Ensemble LSTM + Random Forest Accuracy: 1.0
